# Data Science Challenge



Marvel universe graph: [link](http://bioinfo.uib.es/~joemiro/marvel/porgat.txt)
Graph of all the characters and comic books in which they appeared.
* Which character has the most interactions with others ?
* Which commics contains the most characters ?
* Which character is the most recurrent ?
* Which character associations are the most frequent ? and the least frequent ? 

[marvel wikia dumps](http://s3.amazonaws.com/wikia_xml_dumps/e/en/enmarveldatabase_pages_current.xml.7z)
Some useful info about all marvel characters. Is there any interesting infos to complement the Marvel Universe graph?

In [40]:
#Connexion à la bdd
from py2neo import Graph, authenticate, Relationship
graph = Graph()
authenticate("localhost:7474","neo4j", "admin")

In [41]:
#Creation des noeuds
import csv
with open("vertices.csv","r") as f:
    vertices_data = [(line[0], ''.join(line[1:])) for line in csv.reader(f, delimiter=',')]

for vertice in vertices_data[:6486]:
    graph.cypher.execute('''
    CREATE (c: character {number:{I}, name:{N}})
    RETURN c
    ''', {"I":vertice[0], "N":vertice[1]})
for vertice in vertices_data[6485:]:
    graph.cypher.execute('''
    CREATE (c: comics {number: {I}, name:{N}})
    RETURN c
    ''', {"I":vertice[0], "N":vertice[1]})

In [48]:
#Creation des relations entre character et comics
with open("edges.csv",'r') as f:
    edges_data = [list(line) for line in csv.reader(f,delimiter=',')]

for edges in edges_data:
    for dest in edges[1:]:
        graph.cypher.execute('''
        MATCH (c : character {number: {I}}),
              (x : comics {number: {N}})
        MERGE (c)-[r:APPEARS_IN]->(x)
        RETURN r
        ''', {"I": edges[0], "N": dest})